In [ ]:
import os
from glob import glob
from PIL import Image
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models
import segmentation_models_pytorch as smp
import matplotlib.pyplot as plt

# =============================
# CONFIGURAÇÕES
# =============================
DATA_DIR = '../data/processed'
MODEL_DIR = '../models'
IMAGE_SIZE = (256, 256)
BATCH_SIZE = 8
EPOCHS = 10
LEARNING_RATE = 1e-4
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
NUM_CLASSES = 4

os.makedirs(MODEL_DIR, exist_ok=True)

# =============================
# DATASET
# =============================
class BloodCellDataset(Dataset):
    def __init__(self, images_path, masks_path, transform=None):
        self.images = sorted(glob(f"{images_path}/*/*"))
        self.masks = sorted(glob(f"{masks_path}/*/*"))
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = Image.open(self.images[idx]).convert('RGB').resize(IMAGE_SIZE)
        mask = Image.open(self.masks[idx]).convert('L').resize(IMAGE_SIZE)

        if self.transform:
            image = self.transform(image)

        mask = np.array(mask, dtype=np.float32) / 255.0
        mask = torch.tensor(mask, dtype=torch.float32).unsqueeze(0)

        label_str = os.path.basename(os.path.dirname(self.images[idx]))
        label = {'EOSINOPHIL': 0, 'LYMPHOCYTE': 1, 'MONOCYTE': 2, 'NEUTROPHIL': 3}[label_str]
        label = torch.tensor(label, dtype=torch.long)

        return image, mask, label

# =============================
# TRANSFORMAÇÕES
# =============================
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5],
                         std=[0.5, 0.5, 0.5])
])

# =============================
# DATALOADERS
# =============================
train_dataset = BloodCellDataset(
    images_path=f'{DATA_DIR}/images/train',
    masks_path=f'{DATA_DIR}/masks/train',
    transform=transform
)

val_dataset = BloodCellDataset(
    images_path=f'{DATA_DIR}/images/val',
    masks_path=f'{DATA_DIR}/masks/val',
    transform=transform
)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

# =============================
# MODELOS
# =============================
# 🔸 Segmentação
seg_model = smp.Unet(
    encoder_name="resnet34",
    encoder_weights="imagenet",
    in_channels=3,
    classes=1
).to(DEVICE)

# 🔸 Classificação
clf_model = models.resnet50(pretrained=True)
clf_model.fc = nn.Linear(clf_model.fc.in_features, NUM_CLASSES)
clf_model = clf_model.to(DEVICE)

# =============================
# LOSS FUNCTION & OTIMIZADOR
# =============================
dice_loss = smp.losses.DiceLoss(mode='binary')
pos_weight = torch.tensor([20.0]).to(DEVICE)
focal_loss = smp.losses.FocalLoss(mode='binary')

def seg_loss_fn(logits, target):
    probs = torch.sigmoid(logits)
    print(">>> Epoch", epoch+1, " - Sigmoid probs: min =", probs.min().item(), "max =", probs.max().item())
    return 0.5 * focal_loss(logits, target) + 0.5 * dice_loss(torch.sigmoid(logits), target)

seg_optimizer = torch.optim.Adam(seg_model.parameters(), lr=LEARNING_RATE)
clf_criterion = nn.CrossEntropyLoss()
clf_optimizer = torch.optim.Adam(clf_model.parameters(), lr=LEARNING_RATE)

# =============================
# TREINAMENTO
# =============================
train_seg_loss_history = []
val_seg_loss_history = []
train_clf_loss_history = []
val_clf_loss_history = []

best_val_loss = float('inf')

for epoch in range(EPOCHS):
    seg_model.train()
    clf_model.train()

    total_seg_loss = 0
    total_clf_loss = 0

    for images, masks, labels in train_loader:
        images = images.to(DEVICE)
        masks = masks.to(DEVICE)
        labels = labels.to(DEVICE)

        seg_optimizer.zero_grad()
        seg_outputs = seg_model(images)
        seg_outputs = seg_outputs.squeeze(1)  
        seg_loss = seg_loss_fn(seg_outputs, masks.squeeze(1))
        seg_loss.backward()
        seg_optimizer.step()
        total_seg_loss += seg_loss.item()

        clf_optimizer.zero_grad()
        clf_outputs = clf_model(images)
        clf_loss = clf_criterion(clf_outputs, labels)
        clf_loss.backward()
        clf_optimizer.step()
        total_clf_loss += clf_loss.item()

    avg_train_seg_loss = total_seg_loss / len(train_loader)
    avg_train_clf_loss = total_clf_loss / len(train_loader)
    train_seg_loss_history.append(avg_train_seg_loss)
    train_clf_loss_history.append(avg_train_clf_loss)

    seg_model.eval()
    clf_model.eval()
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    with torch.no_grad():
        for i, (images, masks, labels) in enumerate(val_loader):
            images = images.to(device)
            masks = masks.to(device)

            outputs = seg_model(images)['out']
            probs = torch.sigmoid(outputs)

            # Seleciona um exemplo do batch
            img = images[0].cpu().permute(1, 2, 0).numpy()  # (C, H, W) -> (H, W, C)
            mask = masks[0].cpu().squeeze().numpy()         # (1, H, W) -> (H, W)
            prob = probs[0].cpu().squeeze().numpy()         # (1, H, W) -> (H, W)

            pred_mask = (prob > 0.5).astype(float)

            fig, axs = plt.subplots(1, 4, figsize=(16, 5))
            axs[0].imshow(img, cmap='gray')
            axs[0].set_title('Imagem original')
            axs[1].imshow(mask, cmap='gray')
            axs[1].set_title('Máscara real')
            axs[2].imshow(prob, cmap='viridis')
            axs[2].set_title('Probabilidades sigmoid')
            axs[3].imshow(pred_mask, cmap='gray')
            axs[3].set_title('Previsão binária (>0.5)')
            for ax in axs:
                ax.axis('off')
            plt.tight_layout()
            plt.show()

            break 
        
    val_seg_loss = 0
    val_clf_loss = 0

    with torch.no_grad():
        for images, masks, labels in val_loader:
            images = images.to(DEVICE)
            masks = masks.to(DEVICE)
            labels = labels.to(DEVICE)

            seg_outputs = seg_model(images)
            seg_outputs = seg_outputs.squeeze(1)
            seg_loss = seg_loss_fn(seg_outputs, masks.squeeze(1))
            val_seg_loss += seg_loss.item()

            clf_outputs = clf_model(images)
            clf_loss = clf_criterion(clf_outputs, labels)
            val_clf_loss += clf_loss.item()

    avg_val_seg_loss = val_seg_loss / len(val_loader)
    avg_val_clf_loss = val_clf_loss / len(val_loader)
    val_seg_loss_history.append(avg_val_seg_loss)
    val_clf_loss_history.append(avg_val_clf_loss)

    print(f"[Epoch {epoch+1}/{EPOCHS}] "
          f"Train Seg Loss: {avg_train_seg_loss:.4f} | Val Seg Loss: {avg_val_seg_loss:.4f} || "
          f"Train Clf Loss: {avg_train_clf_loss:.4f} | Val Clf Loss: {avg_val_clf_loss:.4f}")

    if avg_val_seg_loss + avg_val_clf_loss < best_val_loss:
        best_val_loss = avg_val_seg_loss + avg_val_clf_loss
        torch.save(seg_model.state_dict(), os.path.join(MODEL_DIR, 'best_segmentation_model.pth'))
        torch.save(clf_model.state_dict(), os.path.join(MODEL_DIR, 'best_classification_model.pth'))

# =============================
# GRÁFICOS DE LOSS
# =============================
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(train_seg_loss_history, label='Train Seg Loss')
plt.plot(val_seg_loss_history, label='Val Seg Loss')
plt.title('Segmentação - Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(train_clf_loss_history, label='Train Clf Loss')
plt.plot(val_clf_loss_history, label='Val Clf Loss')
plt.title('Classificação - Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.savefig(os.path.join(MODEL_DIR, 'loss_curves.png'))
plt.show()

>>> Epoch 1  - Sigmoid probs: min = 0.014946958050131798 max = 0.999927282333374
>>> Epoch 1  - Sigmoid probs: min = 0.018586551770567894 max = 0.9998716115951538
>>> Epoch 1  - Sigmoid probs: min = 0.0154981454834342 max = 0.9998981952667236
>>> Epoch 1  - Sigmoid probs: min = 0.01954098418354988 max = 0.9999986886978149
>>> Epoch 1  - Sigmoid probs: min = 0.010898507200181484 max = 1.0
>>> Epoch 1  - Sigmoid probs: min = 0.008763106539845467 max = 0.9999991655349731
>>> Epoch 1  - Sigmoid probs: min = 0.024759681895375252 max = 0.9999939203262329
>>> Epoch 1  - Sigmoid probs: min = 0.015749065205454826 max = 0.9999996423721313
>>> Epoch 1  - Sigmoid probs: min = 0.004236746113747358 max = 0.9999997615814209
>>> Epoch 1  - Sigmoid probs: min = 0.005000395234674215 max = 0.9999948740005493
>>> Epoch 1  - Sigmoid probs: min = 0.01150839775800705 max = 0.9999951124191284
>>> Epoch 1  - Sigmoid probs: min = 0.02012430690228939 max = 0.9999998807907104
>>> Epoch 1  - Sigmoid probs: min = 0

ValueError: too many values to unpack (expected 2)

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
from skimage.measure import label, regionprops
from cellpose import models

model = models.CellposeModel(gpu=True, model_type='cyto')

ROOT_DIR = '../data/processed/images'
SPLITS = ['train', 'val', 'test']
CLASSES = ['EOSINOPHIL', 'LYMPHOCYTE', 'MONOCYTE', 'NEUTROPHIL']

all_features = []

print(f"Iniciando segmentação e extração de features...")

for split in SPLITS:
    for cell_class in CLASSES:
        folder = os.path.join(ROOT_DIR, split, cell_class)
        image_files = [f for f in os.listdir(folder) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.tif'))]

        print(f"> Processando {len(image_files)} imagens de {cell_class} [{split}]...")

        for img_file in tqdm(image_files):
            img_path = os.path.join(folder, img_file)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

            if img is None:
                print(f"Erro ao ler {img_path}")
                continue

            masks, flows, styles = model.eval([img], diameter=None, channels=[0, 0], do_3D=False)
            mask = masks[0]

            regions = regionprops(label(mask))
            for i, region in enumerate(regions):
                props = {
                    'split': split,
                    'class': cell_class,
                    'image': img_file,
                    'cell_id': i,
                    'area': region.area,
                    'perimeter': region.perimeter,
                    'eccentricity': region.eccentricity,
                    'solidity': region.solidity,
                    'extent': region.extent,
                    'orientation': region.orientation,
                    'major_axis_length': region.major_axis_length,
                    'minor_axis_length': region.minor_axis_length
                }
                all_features.append(props)

df = pd.DataFrame(all_features)
df.to_csv('cell_features.csv', index=False)

print("\nConcluído! Features salvas em: cell_features.csv")


model_type argument is not used in v4.0.1+. Ignoring this argument...


🚀 Iniciando segmentação e extração de features...
> Processando 5243 imagens de EOSINOPHIL [train]...


  1%|          | 56/5243 [01:40<2:34:34,  1.79s/it]


KeyboardInterrupt: 

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder
import os
import joblib

df = pd.read_csv('cell_features.csv')

feature_cols = [
    'area', 'perimeter', 'eccentricity', 'solidity',
    'extent', 'orientation', 'major_axis_length', 'minor_axis_length'
]

X = df[feature_cols]
y = df['class']

le = LabelEncoder()
y_encoded = le.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

clf = RandomForestClassifier(n_estimators=100, random_state=42)

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print("=== Relatório de Classificação ===")
print(classification_report(y_test, y_pred, target_names=le.classes_))
print(f"Acurácia: {accuracy_score(y_test, y_pred):.4f}")

os.makedirs('../models', exist_ok=True)

joblib.dump(clf, '../models/cell_classifier_rf.joblib')
joblib.dump(le, '../models/label_encoder.joblib')

print("\nModelo treinado e salvo em '../models/'")
